# Deep Learning
```{admonition} Revised
31 May 2023
```

---

## Programming Environment

In [1]:
import numpy             as np
np.set_printoptions(suppress=True, formatter={'float_kind' : '{:0.8f}'.format})
import pandas            as pd
pd.set_option('display.float_format', lambda x: f'{x:0.8f}')
import matplotlib        as mpl
import matplotlib.pyplot as plt

import sklearn
from   sklearn.model_selection import train_test_split
from   sklearn.preprocessing   import OneHotEncoder, StandardScaler

import tensorflow as tf
from   tensorflow.keras.layers import Dense
from   tensorflow.keras.models import Sequential

from   pathlib  import Path

import datetime
from   importlib.metadata import version
import os
import platform as p
import sys

pad = 20
print(  f"\n{'Executed' : <{pad}} : {datetime.datetime.now().astimezone().strftime('%Y-%m-%d %H:%M:%S %z %Z')}"
        f"\n{'Platform' : <{pad}} : {p.platform(aliased = False, terse = False)}"
        f"\n{'Conda'    : <{pad}} : {os.environ['CONDA_DEFAULT_ENV'] or sys.executable.split('/')[-3]}"
        f"\n{'Python'   : <{pad}} : {p.python_implementation()} {p.python_version()} {sys.executable}")
print(*[f'{name : <{pad}} : {version(name)}'
        for name in ['Matplotlib', 'NumPy', 'Pandas', 'Scikit-Learn', 'TensorFlow-macos', 'TensorFlow-metal']], sep = '\n')


Executed             : 2023-09-04 17:37:51 -0400 EDT
Platform             : macOS-13.5.1-arm64-arm-64bit
Conda                : tf
Python               : CPython 3.11.5 /Users/df/anaconda3/envs/tf/bin/python
Matplotlib           : 3.7.2
NumPy                : 1.24.3
Pandas               : 2.1.0
Scikit-Learn         : 1.3.0
TensorFlow-macos     : 2.13.0
TensorFlow-metal     : 1.0.1


---

In [2]:
applicant_data_df = pd.read_csv(filepath_or_buffer='data/applicants_data.csv')
applicant_data_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
applicant_data_df.dtypes

EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [4]:
applicant_data_df = applicant_data_df.drop(columns=['EIN','NAME'])
applicant_data_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
categorical_variables = list(applicant_data_df.dtypes[applicant_data_df.dtypes=='object'].index)
categorical_variables

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [6]:
pd.reset_option('display.float_format')

enc        = OneHotEncoder(sparse_output=False)
encoded_df = pd.DataFrame(data   =enc.fit_transform(X=applicant_data_df[categorical_variables]),
                          columns=enc.get_feature_names_out(input_features=categorical_variables))
encoded_df = pd.concat([
  applicant_data_df[['STATUS','ASK_AMT','IS_SUCCESSFUL']],
  encoded_df,
], axis=1)
encoded_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [7]:
X = encoded_df.drop(columns=['IS_SUCCESSFUL'])
y = encoded_df['IS_SUCCESSFUL']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

scaler         = StandardScaler()
X_scaler       = scaler.fit(X=X_train)
X_train_scaled = X_scaler.transform(X=X_train)
X_test_scaled  = X_scaler.transform(X=X_test)

In [8]:
number_input_features = len(X_train.iloc[0])
print(number_input_features)
number_output_neurons = 1
hidden_nodes_layer1   = (number_input_features + number_output_neurons) // 2
print(hidden_nodes_layer1)
hidden_nodes_layer2   = (hidden_nodes_layer1   + number_output_neurons) // 2
print(hidden_nodes_layer2)

nn = Sequential()
nn.add(Dense(units     =hidden_nodes_layer1,
             input_dim =number_input_features,
             activation='relu'))
nn.add(Dense(units     =hidden_nodes_layer2,
             activation='relu'))
nn.add(Dense(units     =number_output_neurons,
             activation='sigmoid'))
print(nn.summary())

nn.compile(
  loss     ='binary_crossentropy',
  optimizer='adam',
  metrics  =['accuracy'],
)

fit_model = nn.fit(
  x     =X_train_scaled,
  y     =y_train,
  epochs=50,
)

116
58
29
Metal device set to: Apple M1 Max

systemMemory: 64.00 GB
maxCacheSize: 24.00 GB

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 58)                6786      
                                                                 
 dense_1 (Dense)             (None, 29)                1711      
                                                                 
 dense_2 (Dense)             (None, 1)                 30        
                                                                 
Total params: 8,527
Trainable params: 8,527
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50


2023-05-31 16:32:59.753876: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


804/804 [==============================] - 6s 7ms/step - loss: 0.6146 - accuracy: 0.7035
Epoch 2/50
804/804 [==============================] - 6s 7ms/step - loss: 0.5949 - accuracy: 0.7168
Epoch 3/50
804/804 [==============================] - 6s 7ms/step - loss: 0.6161 - accuracy: 0.7015
Epoch 4/50
804/804 [==============================] - 6s 7ms/step - loss: 0.6362 - accuracy: 0.6919
Epoch 5/50
804/804 [==============================] - 6s 7ms/step - loss: 0.6805 - accuracy: 0.6842
Epoch 6/50
804/804 [==============================] - 6s 7ms/step - loss: 0.6880 - accuracy: 0.6737
Epoch 7/50
804/804 [==============================] - 6s 7ms/step - loss: 0.7300 - accuracy: 0.6657
Epoch 8/50
804/804 [==============================] - 6s 7ms/step - loss: 0.7589 - accuracy: 0.6571
Epoch 9/50
804/804 [==============================] - 6s 7ms/step - loss: 0.7596 - accuracy: 0.6627
Epoch 10/50
804/804 [==============================] - 6s 7ms/step - loss: 0.7565 - accuracy: 0.6552
Epoch 11/5

### Optimization

#### First revision - adding another layer

In [9]:
number_input_features    = len(X_train.iloc[0])
print(number_input_features)
number_output_neurons_A1 = 1
hidden_nodes_layer1_A1   = (number_input_features  + number_output_neurons_A1) // 2
print(hidden_nodes_layer1_A1)
hidden_nodes_layer2_A1   = (hidden_nodes_layer1_A1 + number_output_neurons_A1) // 2
print(hidden_nodes_layer2_A1)
hidden_nodes_layer3_A1   = (hidden_nodes_layer2_A1 + number_output_neurons_A1) // 2
print(hidden_nodes_layer3_A1)

nn_A1 = Sequential()
nn_A1.add(Dense(units     =hidden_nodes_layer1_A1,
                input_dim =number_input_features,
                activation='relu'))
nn_A1.add(Dense(units     =hidden_nodes_layer2_A1,
                activation='relu'))
nn_A1.add(Dense(units     =hidden_nodes_layer3_A1,
                activation='relu'))
nn_A1.add(Dense(units     =number_output_neurons_A1,
                activation='sigmoid'))
print(nn_A1.summary())

nn_A1.compile(
  loss     ='binary_crossentropy',
  optimizer='adam',
  metrics  =['accuracy'],
)

fit_model_A1 = nn_A1.fit(
  x     =X_train_scaled,
  y     =y_train,
  epochs=50,
)

116
58
29
15
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 58)                6786      
                                                                 
 dense_4 (Dense)             (None, 29)                1711      
                                                                 
 dense_5 (Dense)             (None, 15)                450       
                                                                 
 dense_6 (Dense)             (None, 1)                 16        
                                                                 
Total params: 8,963
Trainable params: 8,963
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50
804/804 [==============================] - 6s 7ms/step - loss: 0.6397 - accuracy: 0.6941
Epoch 2/50
804/804 [==============================] - 6s 7ms/step 

#### Second revision - 

In [10]:
number_input_features    = len(X_train.iloc[0])
print(number_input_features)
number_output_neurons_A2 = 1
hidden_nodes_layer1_A2   = round(number_input_features * .6666666, 0)
print(hidden_nodes_layer1_A2)
hidden_nodes_layer2_A2   = (hidden_nodes_layer1_A2 + number_output_neurons_A2) // 2
print(hidden_nodes_layer2_A2)

nn_A2 = Sequential()
nn_A2.add(Dense(units     =hidden_nodes_layer1_A2,
                input_dim =number_input_features,
                activation='sigmoid'))
nn_A2.add(Dense(units     =hidden_nodes_layer2_A2,
                activation='sigmoid'))
nn_A2.add(Dense(units     =number_output_neurons_A2,
                activation='sigmoid'))
print(nn_A2.summary())

nn_A2.compile(
  loss     ='binary_crossentropy',
  optimizer='adam',
  metrics  =['accuracy'],
)

fit_model_A2 = nn_A2.fit(
  x     =X_train_scaled,
  y     =y_train,
  epochs=200,
)

116
77.0
39.0
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 77)                9009      
                                                                 
 dense_8 (Dense)             (None, 39)                3042      
                                                                 
 dense_9 (Dense)             (None, 1)                 40        
                                                                 
Total params: 12,091
Trainable params: 12,091
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
804/804 [==============================] - 6s 7ms/step - loss: 0.5909 - accuracy: 0.7055
Epoch 2/200
804/804 [==============================] - 5s 7ms/step - loss: 0.5710 - accuracy: 0.7248
Epoch 3/200
804/804 [==============================] - 5s 6ms/step - loss: 0.5651 - accuracy:

### Model Evaluation

In [13]:
print('Original model results')
model_loss, model_accuracy = nn.evaluate(x=X_test_scaled, y=y_test, verbose=2)
print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

print()
print('First revision model results')
model_loss, model_accuracy = nn_A1.evaluate(x=X_test_scaled, y=y_test, verbose=2)
print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

print()
print('Second revision model results')
model_loss, model_accuracy = nn_A2.evaluate(x=X_test_scaled, y=y_test, verbose=2)
print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

Original model results
268/268 - 1s - loss: 4.8082 - accuracy: 0.6995 - 976ms/epoch - 4ms/step
Loss: 4.808226585388184, Accuracy: 0.6994752287864685

First revision model results
268/268 - 1s - loss: 8585.9287 - accuracy: 0.6152 - 989ms/epoch - 4ms/step
Loss: 8585.9287109375, Accuracy: 0.615160346031189

Second revision model results
268/268 - 1s - loss: 0.5681 - accuracy: 0.7303 - 1s/epoch - 4ms/step
Loss: 0.5680996775627136, Accuracy: 0.7302623987197876


### Save models as HDF5

In [12]:
nn   .save(filepath='data/AlphabetSoup.h5')
nn_A1.save(filepath='data/AlphabetSoup_A1.h5')
nn_A2.save(filepath='data/AlphabetSoup_A2.h5')

---